# Librerias

In [1]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from sqlalchemy import create_engine
import json
import psycopg2
from psycopg2.extras import execute_batch
import numpy as np
import re
import unicodedata
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Funciones

## Conexion a Base de datos

In [2]:
def conection_bd():
    try:
        host = "directob2b.postgres.database.azure.com"
        port = "5432"
        database = "ecommerce"
        user = "directob2b"
        password = "Noctua2025$"
        engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
        print("Conexión exitosa 🚀")
        return engine
    except Exception as e:
        print("Error al conectar con SQLAlchemy:", e)
        return None

## Datos Produtos Arbol

In [3]:
engine = conection_bd()
try:
    query = """SELECT tp.csku,
       tp.cnombre,
       tp.cdescripcion,
       tp.cmarca,
       tnc."id" as id_categoria,
       tnc.categoria
FROM tbl_producto as tp
  LEFT JOIN tbl_nuevas_categorias as tnc
    ON tnc."id" = tp.id_nueva_categoria"""
    df_productos = pd.read_sql(query, engine)
    print('Catálogo obtenido')
except Exception as e:
    print("Error al ejecutar la consulta:", e)


engine.dispose()

Conexión exitosa 🚀
Catálogo obtenido


## Ver prodcutos

In [4]:
df_productos.sample(5)

,csku,cnombre,cdescripcion,cmarca,id_categoria,categoria
3075,006R01817,"TONER XEROX 006R01817, METERED, PARA VERSAINK","TONER XEROX 006R01817, METERED, PARA VERSAINK",Xerox,3.0,Impresion
30659,7722,Pistola de silicón santul de 12w,Pistolas eléctricas para silicón de 12w. diáme...,Santul,5.0,Otros
50111,106830,Paquete de hojas de color saira tamaño c,Papel bond Color verde limón Tamaño carta 21....,Sajor,4.0,Papeleria
16372,LT-ACC-TST,Botella de prueba funcional de carbonato de di...,Botella de prueba funcional de carbonato de di...,Xtralis,NaN,None
70613,X62G,"Mini Burbuja de LED Serie X62, Color Verde","Mini Burbuja de LED Serie X62, Color Verde",Epcom,NaN,None


## Normlaiza texto:

In [5]:

# --- Normalización igual que en entrenamiento ---
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

# Uso Red

# Nuevas Categorias

In [6]:
# Crear el DataFrame
nuevas_categorias = pd.DataFrame({
    'id': range(1, 6),  # IDs del 1 al 5
    'categoria': ['computo', 'video vigilancia', 'impresion', 'papeleria', 'otros']
})

# Mostrar el DataFrame
print(nuevas_categorias)

   id         categoria
0   1           computo
1   2  video vigilancia
2   3         impresion
3   4         papeleria
4   5             otros


# Cargar Modelo

In [7]:
# Cargar el modelo guardado
model = tf.keras.models.load_model("modelo_categorias.keras")

# Ver datos

In [8]:
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Normalización igual que en entrenamiento ---
def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

# Cargar la base original para reconstruir tokenizer y LabelEncoder
df_datos = df_productos.copy()
# Normalizar columnas
for col in ['cnombre', 'cdescripcion', 'cmarca']:
    df_datos[col] = df_datos[col].fillna('').astype(str).apply(normalize_text)

# Separar etiquetados
df_labeled = df_datos[df_datos['categoria'].notna()].copy()
df_labeled['texto'] = df_labeled['cnombre'] + ' ' + df_labeled['cdescripcion'] + ' ' + df_labeled['cmarca']

# Reconstruir tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_labeled['texto'])

# Reconstruir LabelEncoder
le = LabelEncoder()
le.fit(df_labeled['categoria'])


LabelEncoder()

In [9]:
import pickle

# Guardar
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

with open("labelencoder.pkl", "wb") as f:
    pickle.dump(le, f)

# FUncion

In [10]:
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from unicodedata import normalize
import re


In [11]:
def normalize_text(text):
    text = text.lower()
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

In [12]:
# Cargar modelo
model = tf.keras.models.load_model("modelo_categorias.keras")

In [13]:
# Cargar Tokenizer y LabelEncoder
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("labelencoder.pkl", "rb") as f:
    le = pickle.load(f)


In [14]:
# Ejemplo: predecir categoría para nuevo texto
nuevo_texto = ["Laptop HP 14 pulgadas procesador intel core i5"]

In [15]:

def normalize_text(text):
    text = text.lower()
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

In [16]:
# Normalizar y convertir a secuencia
texto_norm = [normalize_text(t) for t in nuevo_texto]
seq = tokenizer.texts_to_sequences(texto_norm)
X_new = pad_sequences(seq, maxlen=200)

In [17]:
# Predecir
y_pred = model.predict(X_new)
categoria_pred = le.inverse_transform(np.argmax(y_pred, axis=1))
print(categoria_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
['Computo']


In [18]:
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71354 entries, 0 to 71353
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   csku          71354 non-null  object 
 1   cnombre       71354 non-null  object 
 2   cdescripcion  71354 non-null  object 
 3   cmarca        71347 non-null  object 
 4   id_categoria  49339 non-null  float64
 5   categoria     49339 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.3+ MB
